In [1]:
import logging
import argparse
import os
import numpy as np
import torch

from ppnp.pytorch import PPNP
from ppnp.pytorch.training import train_model
from ppnp.pytorch.earlystopping import stopping_args
from ppnp.pytorch.propagation import PPRExact, PPRPowerIteration
from ppnp.data.io import load_dataset

In [2]:
logging.basicConfig(
        format='%(asctime)s: %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO + 2)

# PyTorch variant
This is not the official code that should be used to reproduce the results from the paper (see `reproduce_results.ipynb` for this), but an adaptation of that code to PyTorch for better accessibility. This notebook reproduces the accuracy of the TensorFlow implementation, but has a longer computation time and varies in some details due to the change to PyTorch.

# Load dataset

In [3]:
graph_name = 'cora_ml'
graph = load_dataset(graph_name)
graph.standardize(select_lcc=True)

<Undirected, unweighted and connected SparseGraph with 15962 edges (no self-loops). Data: adj_matrix (2810x2810), attr_matrix (2810x2879), labels (2810), node_names (2810), attr_names (2879), class_names (7)>

# Set up data splits

First of all we need to decide whether to use the test or validation set. Be mindful that we can only look at the test set exactly _once_ and then can't change any hyperparameters oder model details, no matter what. Everything else would cause overfitting.

In [4]:
test = True

These are the seeds for the dataset splits used in the paper for test/validation.

In [5]:
test_seeds = [
        2144199730,  794209841, 2985733717, 2282690970, 1901557222,
        2009332812, 2266730407,  635625077, 3538425002,  960893189,
        497096336, 3940842554, 3594628340,  948012117, 3305901371,
        3644534211, 2297033685, 4092258879, 2590091101, 1694925034]
val_seeds = [
        2413340114, 3258769933, 1789234713, 2222151463, 2813247115,
        1920426428, 4272044734, 2092442742, 841404887, 2188879532,
        646784207, 1633698412, 2256863076,  374355442,  289680769,
        4281139389, 4263036964,  900418539,  119332950, 1628837138]

if test:
    seeds = test_seeds
else:
    seeds = val_seeds

Now we can choose the remaining settings for the training/early stopping/validation(test) split. These are the ones chosen in the paper

In [6]:
if graph_name == 'microsoft_academic':
    nknown = 5000
else:
    nknown = 1500
    
idx_split_args = {'ntrain_per_class': 20, 'nstopping': 500, 'nknown': nknown}

# Set up propagation

Next we need to set up the proper pmropagation scheme. In the paper we've introduced the exact PPR propagation used in PPNP and the PPR power iteration propagation used in APPNP.

We use the hyperparameters from the paper.

In [7]:
if graph_name == 'microsoft_academic':
    alpha = 0.2
else:
    alpha = 0.1

prop_ppnp = PPRExact(graph.adj_matrix, alpha=alpha)
prop_appnp = PPRPowerIteration(graph.adj_matrix, alpha=alpha, niter=10)

# Choose model hyperparameters

Now we choose the hyperparameters. These are the ones used in the paper for all datasets.

Note that we choose the propagation for APPNP.

In [8]:
model_args = {
    'hiddenunits': [64],
    'drop_prob': 0.5,
    'propagation': prop_appnp}

reg_lambda = 5e-3
learning_rate = 0.01

# Train model

First we set the remaining settings for training.

In [9]:
niter_per_seed = 5
save_result = False
print_interval = 100
device = 'cuda'

We use 20 different seeds for splitting and 5 iterations (different random initializations) per split, so we train 100 times altogether. This will take a while.

In [10]:
results = []
niter_tot = niter_per_seed * len(seeds)
i_tot = 0
for seed in seeds:
    idx_split_args['seed'] = seed
    for _ in range(niter_per_seed):
        i_tot += 1
        logging_string = f"Iteration {i_tot} of {niter_tot}"
        logging.log(22,
                logging_string + "\n                     "
                + '-' * len(logging_string))
        _, result = train_model(
            graph_name, PPNP, graph, model_args, learning_rate, reg_lambda,
            idx_split_args, stopping_args, test, device, None, print_interval)
        results.append({})
        results[-1]['stopping_accuracy'] = result['early_stopping']['accuracy']
        results[-1]['valtest_accuracy'] = result['valtest']['accuracy']
        results[-1]['runtime'] = result['runtime']
        results[-1]['runtime_perepoch'] = result['runtime_perepoch']
        results[-1]['split_seed'] = seed

2019-12-04 12:35:06: Iteration 1 of 100
                     ------------------
2019-12-04 12:35:06: PyTorch seed: 3735636244
2019-12-04 12:37:01: Last epoch: 1641, best epoch: 1273 (112.727 sec)
2019-12-04 12:37:01: Test accuracy: 83.4%
2019-12-04 12:37:01: Iteration 2 of 100
                     ------------------
2019-12-04 12:37:01: PyTorch seed: 301655472
2019-12-04 12:38:26: Last epoch: 1226, best epoch: 1126 (84.561 sec)
2019-12-04 12:38:26: Test accuracy: 85.6%
2019-12-04 12:38:26: Iteration 3 of 100
                     ------------------
2019-12-04 12:38:26: PyTorch seed: 2898121914
2019-12-04 12:40:33: Last epoch: 1854, best epoch: 1680 (127.663 sec)
2019-12-04 12:40:33: Test accuracy: 84.2%
2019-12-04 12:40:34: Iteration 4 of 100
                     ------------------
2019-12-04 12:40:34: PyTorch seed: 1529719124
2019-12-04 12:43:01: Last epoch: 2153, best epoch: 2053 (147.002 sec)
2019-12-04 12:43:01: Test accuracy: 84.5%
2019-12-04 12:43:01: Iteration 5 of 100
          

# Evaluation

To evaluate the data we use Pandas and Seaborn (for bootstrapping).

In [11]:
import pandas as pd
import seaborn as sns

In [12]:
result_df = pd.DataFrame(results)
result_df.head()

,stopping_accuracy,valtest_accuracy,runtime,runtime_perepoch,split_seed
0,0.832,0.833588,112.726716,0.068652,2144199730
1,0.838,0.855725,84.561042,0.068917,2144199730
2,0.830,0.841985,127.662649,0.068821,2144199730
3,0.838,0.845038,147.002493,0.068246,2144199730
4,0.836,0.845038,97.349578,0.068653,2144199730


The standard deviation doesn't really say much about the uncertainty of our results and the standard error of the mean (SEM) assumes a normal distribution. So the best way to get a valid estimate for our results' uncertainty is via bootstrapping.

In [14]:
def calc_uncertainty(values: np.ndarray, n_boot: int = 1000, ci: int = 95) -> dict:
    stats = {}
    stats['mean'] = values.mean()
    boots_series = sns.algorithms.bootstrap(values, func=np.mean, n_boot=n_boot)
    stats['CI'] = sns.utils.ci(boots_series, ci)
    stats['uncertainty'] = np.max(np.abs(stats['CI'] - stats['mean']))
    return stats

In [15]:
stopping_acc = calc_uncertainty(result_df['stopping_accuracy'])
valtest_acc = calc_uncertainty(result_df['valtest_accuracy'])
runtime = calc_uncertainty(result_df['runtime'])
runtime_perepoch = calc_uncertainty(result_df['runtime_perepoch'])

In [16]:
print("APPNP\n"
      "Early stopping: Accuracy: {:.2f} ± {:.2f}%\n"
      "{}: Accuracy: {:.2f} ± {:.2f}%\n"
      "Runtime: {:.3f} ± {:.3f} sec, per epoch: {:.2f} ± {:.2f}ms"
      .format(
          stopping_acc['mean'] * 100,
          stopping_acc['uncertainty'] * 100,
          'Test' if test else 'Validation',
          valtest_acc['mean'] * 100,
          valtest_acc['uncertainty'] * 100,
          runtime['mean'],
          runtime['uncertainty'],
          runtime_perepoch['mean'] * 1e3,
          runtime_perepoch['uncertainty'] * 1e3,
      ))

APPNP
Early stopping: Accuracy: 84.03 ± 0.40%
Test: Accuracy: 85.07 ± 0.26%
Runtime: 119.764 ± 4.501 sec, per epoch: 68.53 ± 0.14ms
